In [ ]:
from langchain.llms.gpt4all import GPT4All
from langchain.prompts import PromptTemplate

# prompt = PromptTemplate.from_template("A {word} is a")
prompt = PromptTemplate.from_template("You are a helpful assistant that defineds words. Define this wird: {word}.")

llm = GPT4All(model = "gpt4all-falcon-newbpe-q4_0.gguf")

chain = prompt | llm

chain.invoke({"word": "tomato"})
# with model.chat_session():
#     print(model.generate("How can I run LLMs efficiently on my laptop?", max_tokens=1024))

'\nTomato is a fruit that grows on a vine and has a red, juicy flesh. It is often used in cooking and can be eaten raw or cooked.'